### Inventory Management Problem

In [76]:
import numpy as np
import math
import dimod
from neal import SimulatedAnnealingSampler
from inventory import *

### Generating Sample Problem

In [77]:
#n_products = int(input("Number of Products:"))
#n_raw_materials = int(input("Number of Raw Materials:"))

n_products = 3
n_raw_materials = 7
print(f"Generating for {n_products} products and {n_raw_materials} raw materials.")

Generating for 3 products and 7 raw materials.


In [78]:
S = np.array([200., 240., 160.])
A = np.floor(10 * (np.random.random(size=(n_raw_materials, n_products)))) + 1
R = np.array([300., 360., 240., 270., 450., 600., 540.])  
C = np.floor(5 * np.random.random(size=n_raw_materials)) + 1  
D = np.array([30, 30, 30])

S.shape, A.shape, R.shape, C.shape, D.shape

((3,), (7, 3), (7,), (7,), (3,))

In [79]:
P = C @ A - S.T #objective matrix

### Encoding Functions

In [80]:
def bounded_coefficient_encoding(kappa_x, mu_x):
    """Bounded-Coefficient Encoding Algorithm"""
    #print(f"  Encoding variable with κ={kappa_x}, μ={mu_x}")
    
    if kappa_x < 2**(math.floor(math.log2(mu_x)) + 1):
        print(f"    Using binary encoding")
        log_kappa = math.floor(math.log2(kappa_x))
        binary_coeffs = [2**i for i in range(log_kappa + 1)]
        
        if sum(binary_coeffs[:-1]) < kappa_x:
            binary_coeffs[-1] = kappa_x - sum(binary_coeffs[:-1])
        
        print(f"    Coefficients: {binary_coeffs}")
        return binary_coeffs
    else:
        #print(f"    Using bounded-coefficient encoding")
        rho = math.floor(math.log2(mu_x)) + 1
        nu = kappa_x - sum(2**(i-1) for i in range(1, rho + 1))
        eta = math.floor(nu / mu_x)
        
        c_x = []
        for i in range(1, rho + 1):
            c_x.append(2**(i-1))
        for i in range(eta):
            c_x.append(mu_x)
        
        remainder = nu - eta * mu_x
        if remainder != 0:
            c_x.append(remainder)
        
        #print(f"    Coefficients: {c_x}")
        return c_x

def create_bounded_encoding_system(kappa_vec, mu_vec):
    """Create complete encoding system"""
    
    all_coefficients = []
    widths = []
    
    for i in range(len(kappa_vec)):
        coeffs = bounded_coefficient_encoding(kappa_vec[i], mu_vec[i])
        all_coefficients.append(coeffs)
        widths.append(len(coeffs))
    
    total_binary_vars = sum(widths)
    print(f"Total binary variables: {total_binary_vars}")
    
    return all_coefficients, widths, total_binary_vars

def create_bounded_encoding_matrix(n_variables, all_coefficients, widths):
    """Create encoding matrix C where x = C @ y"""
    total_binary_vars = sum(widths)
    C = np.zeros((n_variables, total_binary_vars))
    
    start_idx = 0
    for i in range(n_variables):
        end_idx = start_idx + widths[i]
        C[i, start_idx:end_idx] = all_coefficients[i]
        start_idx = end_idx
    
    return C

In [81]:
kappa_vec = D.astype(int)
mu_vec = [4, 4, 4]
all_coefficients, widths, total_binary_vars = create_bounded_encoding_system(kappa_vec, mu_vec)
C_enc = create_bounded_encoding_matrix(n_products, all_coefficients, widths)

Total binary variables: 27


### QUBO Formulation

In [ ]:
penalty_lambda = 1000

In [83]:
variable_names = []
for i in range(n_products):
    for j in range(widths[i]):
        variable_names.append(f"x{i+1}_bin{j+1}")

In [84]:
q_linear = P.T @ C_enc
q_linear.shape

(27,)

In [85]:
Q = np.zeros((total_binary_vars, total_binary_vars))
q_penalty = np.zeros(total_binary_vars)

In [86]:
for j in range(n_raw_materials):
    A_j = A[j, :].reshape(1, -1)
    Q_j = C_enc.T @ A_j.T @ A_j @ C_enc
    Q += penalty_lambda * Q_j

    q_penalty_j = -2 * penalty_lambda * R[j] * (A_j @ C_enc).flatten()
    q_penalty += q_penalty_j

In [87]:
q = q_linear + q_penalty

In [88]:
Q.shape, q.shape

((27, 27), (27,))

### BQM Formulation

In [89]:
bqm = dimod.BinaryQuadraticModel.empty('BINARY')

In [90]:
for i, var_name in enumerate(variable_names):
    bqm.add_variable(var_name, q[i])

for i in range(len(variable_names)):
    for j in range(i+1, len(variable_names)):
        if abs(Q[i, j]) > 1e-10:
            bqm.add_interaction(variable_names[i], variable_names[j], Q[i,j])

In [91]:
sampler = SimulatedAnnealingSampler()
sampleset = sampler.sample(bqm, num_reads=1000)

In [92]:
def decode_solution(solution, C_enc, variable_names, all_coefficients, widths):
    """Decode binary solution back to original variables"""
    
    y = np.array([solution[var_name] for var_name in variable_names])
    x = C_enc @ y
    
    print(f"Binary solution: {y}")
    print(f"Decoded solution x: {x}")
    
    return x, y

In [93]:
x_optimal, y_optimal = decode_solution(
    sampleset.first.sample, C_enc, variable_names, all_coefficients, widths
)

Binary solution: [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
Decoded solution x: [30. 30. 30.]


### Testing approaches

In [1]:
import numpy as np
import math
import dimod
from neal import SimulatedAnnealingSampler

def bounded_coefficient_encoding(kappa_x, mu_x):
    """Bounded-Coefficient Encoding Algorithm"""
    
    if kappa_x < 2**(math.floor(math.log2(mu_x)) + 1):
        # Use binary encoding
        log_kappa = math.floor(math.log2(kappa_x))
        binary_coeffs = [2**i for i in range(log_kappa + 1)]
        
        if sum(binary_coeffs[:-1]) < kappa_x:
            binary_coeffs[-1] = kappa_x - sum(binary_coeffs[:-1])
        
        return binary_coeffs
    else:
        # Use bounded-coefficient encoding
        rho = math.floor(math.log2(mu_x)) + 1
        nu = kappa_x - sum(2**(i-1) for i in range(1, rho + 1))
        eta = math.floor(nu / mu_x)
        
        c_x = []
        for i in range(1, rho + 1):
            c_x.append(2**(i-1))
        for i in range(eta):
            c_x.append(mu_x)
        
        remainder = nu - eta * mu_x
        if remainder != 0:
            c_x.append(remainder)
        
        return c_x

def create_inventory_bqm(S, A, R, C, D, penalty_strength=1000):
    """
    Create BQM for inventory management problem
    
    Parameters:
    - S: selling prices (n_products,)
    - A: recipe matrix (n_raw_materials, n_products) 
    - R: raw material upper limits (n_raw_materials,)
    - C: raw material costs (n_raw_materials,)
    - D: demand limits (n_products,)
    - penalty_strength: strength of constraint penalties
    
    Returns:
    - bqm: Binary Quadratic Model
    - encoding_info: Information about variable encoding
    """
    
    n_products = len(S)
    n_raw_materials = len(R)
    
    # Calculate objective coefficients
    P = C @ A - S  # Cost of raw materials minus selling price
    
    print(f"Problem size: {n_products} products, {n_raw_materials} raw materials")
    print(f"Objective coefficients P: {P}")
    print(f"Demand limits D: {D}")
    print(f"Raw material limits R: {R}")
    
    # Step 1: Encode production variables x1, x2, ..., xn with bounds D1, D2, ..., Dn
    x_coefficients = []
    x_widths = []
    
    for i in range(n_products):
        coeffs = bounded_coefficient_encoding(int(D[i]), int(D[i]))
        x_coefficients.append(coeffs)
        x_widths.append(len(coeffs))
        print(f"x{i+1} encoding: {len(coeffs)} binary variables, coefficients: {coeffs}")
    
    # Step 2: Encode slack variables s1, s2, ..., sm with bounds R1, R2, ..., Rm
    s_coefficients = []
    s_widths = []
    
    for i in range(n_raw_materials):
        coeffs = bounded_coefficient_encoding(int(R[i]), int(R[i]))
        s_coefficients.append(coeffs)
        s_widths.append(len(coeffs))
        print(f"s{i+1} encoding: {len(coeffs)} binary variables, coefficients: {coeffs}")
    
    # Step 3: Create variable mapping
    total_x_vars = sum(x_widths)
    total_s_vars = sum(s_widths)
    total_vars = total_x_vars + total_s_vars
    
    print(f"Total binary variables: {total_vars} ({total_x_vars} for x, {total_s_vars} for s)")
    
    # Create mapping from (variable_type, index, bit) to binary variable index
    var_map = {}
    current_idx = 0
    
    # Map x variables
    for i in range(n_products):
        for j in range(x_widths[i]):
            var_map[('x', i, j)] = current_idx
            current_idx += 1
    
    # Map s variables  
    for i in range(n_raw_materials):
        for j in range(s_widths[i]):
            var_map[('s', i, j)] = current_idx
            current_idx += 1
    
    # Step 4: Create BQM
    bqm = dimod.BinaryQuadraticModel({}, {}, 0.0, 'BINARY')
    
    # Add linear terms for objective function: P @ x
    for i in range(n_products):
        for j in range(x_widths[i]):
            var_idx = var_map[('x', i, j)]
            coefficient = P[i] * x_coefficients[i][j]
            bqm.add_variable(var_idx, coefficient)
    
    # Step 5: Add constraint penalties: A @ x + s = R
    # For each raw material constraint: sum(A[i,j] * x[j]) + s[i] = R[i]
    
    for i in range(n_raw_materials):  # For each raw material constraint
        
        # Linear terms: A[i,j] * x[j] terms
        linear_terms = {}
        
        # Add A[i,j] * x[j] terms (positive contribution)
        for j in range(n_products):
            for k in range(x_widths[j]):
                var_idx = var_map[('x', j, k)]
                coeff = A[i, j] * x_coefficients[j][k]
                if var_idx in linear_terms:
                    linear_terms[var_idx] += coeff
                else:
                    linear_terms[var_idx] = coeff
        
        # Add s[i] terms (positive contribution)
        for k in range(s_widths[i]):
            var_idx = var_map[('s', i, k)]
            coeff = s_coefficients[i][k]
            if var_idx in linear_terms:
                linear_terms[var_idx] += coeff
            else:
                linear_terms[var_idx] = coeff
        
        # Add quadratic penalty terms for constraint: (A @ x + s - R)^2
        # Expand: (sum_terms - R[i])^2 = sum_terms^2 - 2*R[i]*sum_terms + R[i]^2
        
        # Quadratic terms: interactions between different variables
        var_indices = list(linear_terms.keys())
        for idx1 in range(len(var_indices)):
            for idx2 in range(idx1, len(var_indices)):
                v1, v2 = var_indices[idx1], var_indices[idx2]
                coeff1, coeff2 = linear_terms[v1], linear_terms[v2]
                
                if v1 == v2:
                    # Diagonal term: coeff^2
                    penalty = penalty_strength * coeff1 * coeff2
                    bqm.add_variable(v1, penalty)
                else:
                    # Off-diagonal term: 2 * coeff1 * coeff2
                    penalty = 2 * penalty_strength * coeff1 * coeff2
                    bqm.add_interaction(v1, v2, penalty)
        
        # Linear penalty terms: -2 * R[i] * sum_terms
        for var_idx, coeff in linear_terms.items():
            penalty = -2 * penalty_strength * R[i] * coeff
            bqm.add_variable(var_idx, penalty)
        
        # Constant term: R[i]^2 (added to offset)
        bqm.offset += penalty_strength * R[i] * R[i]
    
    # Encoding information for solution decoding
    encoding_info = {
        'n_products': n_products,
        'n_raw_materials': n_raw_materials,
        'x_coefficients': x_coefficients,
        's_coefficients': s_coefficients,
        'x_widths': x_widths,
        's_widths': s_widths,
        'var_map': var_map,
        'total_vars': total_vars,
        'A': A,
        'R': R,
        'P': P,
        'D': D
    }
    
    return bqm, encoding_info

def decode_solution(sample, encoding_info):
    """Decode binary solution back to original variables"""
    
    x_coeffs = encoding_info['x_coefficients']
    s_coeffs = encoding_info['s_coefficients']
    x_widths = encoding_info['x_widths']
    s_widths = encoding_info['s_widths']
    var_map = encoding_info['var_map']
    n_products = encoding_info['n_products']
    n_raw_materials = encoding_info['n_raw_materials']
    
    # Decode x variables
    x_values = np.zeros(n_products)
    for i in range(n_products):
        for j in range(x_widths[i]):
            var_idx = var_map[('x', i, j)]
            if sample[var_idx] == 1:
                x_values[i] += x_coeffs[i][j]
    
    # Decode s variables
    s_values = np.zeros(n_raw_materials)
    for i in range(n_raw_materials):
        for j in range(s_widths[i]):
            var_idx = var_map[('s', i, j)]
            if sample[var_idx] == 1:
                s_values[i] += s_coeffs[i][j]
    
    return x_values, s_values

def validate_solution(x_values, s_values, encoding_info):
    """Validate that solution satisfies constraints"""
    
    A = encoding_info['A']
    R = encoding_info['R']
    D = encoding_info['D']
    P = encoding_info['P']
    
    print("Solution Validation:")
    print(f"Production quantities: {x_values}")
    print(f"Slack variables: {s_values}")
    
    # Check demand constraints
    demand_satisfied = all(x_values[i] <= D[i] for i in range(len(D)))
    print(f"Demand constraints satisfied: {demand_satisfied}")
    
    # Check raw material constraints
    raw_material_usage = A @ x_values
    constraint_satisfaction = raw_material_usage + s_values
    constraints_satisfied = np.allclose(constraint_satisfaction, R, atol=1e-6)
    
    print(f"Raw material usage: {raw_material_usage}")
    print(f"Raw material limits: {R}")
    print(f"Constraint A@x + s: {constraint_satisfaction}")
    print(f"Should equal R: {R}")
    print(f"Raw material constraints satisfied: {constraints_satisfied}")
    
    # Calculate objective value
    objective_value = P @ x_values
    print(f"Objective value: {objective_value}")
    
    return demand_satisfied and constraints_satisfied

# Example usage
if __name__ == "__main__":
    # Problem setup (using your data)
    n_products = 3
    n_raw_materials = 5  # Note: you mentioned 5 raw materials
    
    S = np.array([200., 240., 160.])
    A = np.array([[7., 3., 8.],
                  [2., 9., 4.],
                  [5., 1., 6.],
                  [8., 7., 2.],
                  [3., 5., 9.]])  # 5x3 matrix
    R = np.array([300., 360., 240., 270., 450.])  # 5 raw materials
    C = np.array([3., 2., 4., 1., 5.])  # 5 raw material costs
    D = np.array([30, 30, 30])
    
    print("=== Inventory Management BQM Formulation ===")
    
    # Create BQM
    bqm, encoding_info = create_inventory_bqm(S, A, R, C, D, penalty_strength=1000)
    
    print(f"\nBQM created with {len(bqm.variables)} variables")
    print(f"Linear terms: {len(bqm.linear)}")
    print(f"Quadratic terms: {len(bqm.quadratic)}")
    
    # Solve using simulated annealing
    print("\n=== Solving with Simulated Annealing ===")
    sampler = SimulatedAnnealingSampler()
    sampleset = sampler.sample(bqm, num_reads=100)
    
    best_sample = sampleset.first.sample
    best_energy = sampleset.first.energy
    
    print(f"Best energy: {best_energy}")
    
    # Decode solution
    x_values, s_values = decode_solution(best_sample, encoding_info)
    
    # Validate solution
    is_valid = validate_solution(x_values, s_values, encoding_info)
    print(f"\nSolution is valid: {is_valid}")

=== Inventory Management BQM Formulation ===
Problem size: 3 products, 5 raw materials
Objective coefficients P: [-132. -177.  -57.]
Demand limits D: [30 30 30]
Raw material limits R: [300. 360. 240. 270. 450.]
x1 encoding: 5 binary variables, coefficients: [1, 2, 4, 8, 15]
x2 encoding: 5 binary variables, coefficients: [1, 2, 4, 8, 15]
x3 encoding: 5 binary variables, coefficients: [1, 2, 4, 8, 15]
s1 encoding: 9 binary variables, coefficients: [1, 2, 4, 8, 16, 32, 64, 128, 45]
s2 encoding: 9 binary variables, coefficients: [1, 2, 4, 8, 16, 32, 64, 128, 105]
s3 encoding: 8 binary variables, coefficients: [1, 2, 4, 8, 16, 32, 64, 113]
s4 encoding: 9 binary variables, coefficients: [1, 2, 4, 8, 16, 32, 64, 128, 15]
s5 encoding: 9 binary variables, coefficients: [1, 2, 4, 8, 16, 32, 64, 128, 195]
Total binary variables: 59 (15 for x, 44 for s)

BQM created with 59 variables
Linear terms: 59
Quadratic terms: 937

=== Solving with Simulated Annealing ===
Best energy: -6099.0
Solution Valid